In [1]:
import numpy as np
import pandas as pd
import numpy as np
import sketches
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
from pathlib import Path
import itertools
from multiprocessing import Pool, cpu_count

trial=1
xcol = "space_final"
ycol = "f_error_final"

def get_clean_name(codename):
    translator = {"theirs": "Theirs",
                  "bn-8-HsuRNN-True-ckpts-forwards-more": "BatchRank (K = 8)",
                  "bn-64-HsuRNN-True-ckpts-forwards-more": "BatchRank (K = 64)",
                  "l1-HsuRNN-False-ckpts-forwards-more": "Unweighted L1 Loss",
                  "l1-HsuRNN-True-ckpts-forwards-more": "Weighted L1 Loss",
                  "log_mse-HsuRNN-False-ckpts-forwards-more": "Unweighted Log Loss",
                  "log_mse-HsuRNN-True-ckpts-forwards-more": "Weighted Log Loss",                 
                 }
    return translator[codename]

for instantiation in ["ours"]:
    for formula in ["std"]:
        for minute in [8, 29, 59]:
            for fixed_width in [1000, 3000, 10000, 30000, 100000]:
                a = np.load(f"equinix-chicago.dirA.20160121-13{minute:02d}00.ports.npy", allow_pickle=True)
                sm = a.item()['y'].sum()
                for sketch_type in ["cm"]:
                    A = None
                    for method in ["bn-8-HsuRNN-True-ckpts-forwards-more", "bn-64-HsuRNN-True-ckpts-forwards-more", "l1-HsuRNN-False-ckpts-forwards-more",
                          "l1-HsuRNN-True-ckpts-forwards-more", "log_mse-HsuRNN-False-ckpts-forwards-more", "log_mse-HsuRNN-True-ckpts-forwards-more"]: 
                        exp_path = f"all_logs/{method}/trial{trial}/lightning_logs/minute{minute:02d}_{formula}_giant_results.ftr"
                        df_result = pd.read_feather(exp_path)
                        df_result = df_result.groupby(["width", "nhashes", "perc"]).agg({"f_error": ["mean", "std"], "sum": ["mean"], "space": ["mean"]}).reset_index()
                        df_result["f_error_final"] = df_result["f_error"]["mean"]
                        df_result["space_final"] = df_result["space"]["mean"] 
                        df_result["sum_final"] = df_result["sum"]["mean"]
                        df_result["ar_final"] = df_result["f_error_final"] * sm / df_result["sum_final"]
                        df_result["theory_ar_final"] = df_result["sum_final"] / df_result["width"]
                        A = df_result.copy()
                        B = None
                        for their_method in ["log_mse-HsuRNN-False-ckpts-forwards-more"]:
                            if instantiation == "theirs":
                                exp_path_theirs = f"all_logs/theirs/minute{minute:02d}_{formula}_small_results.ftr"
                            if instantiation == "ours":
                                exp_path_theirs = f"all_logs{their_method}/trial{trial}/lightning_logs/minute{minute:02d}_{formula}_giant_results.ftr"

                            df_result = pd.read_feather(exp_path_theirs)
                            df_result = df_result.groupby(["width", "nhashes", "perc"]).agg({"f_error": ["mean", "std"], "sum": ["mean"], "space": ["mean"]}).reset_index()
                            df_result["f_error_final"] = df_result["f_error"]["mean"]
                            df_result["space_final"] = df_result["space"]["mean"]
                            df_result["sum_final"] = df_result["sum"]["mean"]
                            df_result["ar_final"] = df_result["f_error_final"] * sm / df_result["sum_final"]
                            df_result["theory_ar_final"] = df_result["sum_final"] / df_result["width"]
                            B = df_result.copy()
                            plt.figure(figsize=(5, 5))
                            A = A[A.width == fixed_width]
                            B = B[B.width == fixed_width]
                            plt.plot(B[B.nhashes == 1]["perc"], B[B.nhashes == 1]["sum_final"] / A[A.nhashes == 1]["sum_final"], label = "cc ratio")
                            for fixed_hashes in [1, 2, 3, 4]:
                                def get_descriptor(i):
                                    if i == 1:
                                        return "1 hash"
                                    if i > 1:
                                        return f"{i} hashes"
                                C = A[A.nhashes == fixed_hashes]
                                D = B[B.nhashes == fixed_hashes]
                                plt.plot(D["perc"], D["f_error_final"] / C["f_error_final"], label = f"error ratio ({get_descriptor(fixed_hashes)})")

                            plt.legend()
                            plt.title(f"Minute {minute + 1}, {get_clean_name(method)}, width {fixed_width}")
                            plt.xlabel("Explicit Counter Percentage (%)")
                            plt.ylabel("Ratio (Theirs / Ours)")
                            plt.savefig(f"perfmin{method}{minute:02d}{formula}-{instantiation}-width{fixed_width}.png")
                            plt.show()    


ModuleNotFoundError: No module named 'sketches'